# Add Overrides to Train FERC-EIA Connecter

The FERC-EIA record linkage process requries training data in order to work properly. Training matches also serve as overrides. This notebook helps you check whether the machine learning algroythem did a good job of matching FERC and EIA records. If you find a good match (or you correct a bad match), this process will turn it into training data.

This notebook has two purposes: 

1) [**Output override tools to verify connection between EIA and FERC1**](#verify-tools)
2) [**Upload changes to training data**](#upload-overrides)

## Settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pudl_rmi
from pudl_rmi.create_override_spreadsheets import *
                                           
import pudl
import sqlalchemy as sa
import logging
import sys

import warnings
warnings.filterwarnings('ignore')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq='AS',fill_fuel_cost=True,roll_fuel_cost=True,fill_net_gen=True)
rmi_out = pudl_rmi.coordinate.Output(pudl_out)

## Specify Utilities & Years

In [3]:
# old

specified_utilities = {
    # 'Dominion': {'utility_id_pudl': [292, 293, 349],
    #              'utility_id_eia': [17539, 17554, 19876]},
    # 'Evergy': {'utility_id_pudl': [159, 160, 161, 1270, 13243],
    #            'utility_id_eia': [10000, 10005, 56211, 3702, 55329]}, # pudl/eia 359/22500 --> 13243/55329, 1270/3702 --> BAD
    # 'IDACORP': {'utility_id_pudl': [140],
    #             'utility_id_eia': [9191]},
    # 'Duke': {'utility_id_pudl': [90, 91, 92, 93, 96, 97],
    #          'utility_id_eia': [5416, 6455, 15470, 55729, 3542, 3046]},
    'BHE': {'utility_id_pudl': [185, 246, 204, 287],
            'utility_id_eia': [12341, 14354, 13407, 17166]},
    'Southern': {'utility_id_pudl': [123, 18, 190, 11830],
                 'utility_id_eia': [7140, 195, 12686, 17622]},
    # 'NextEra': {'utility_id_pudl': [121, 130],
    #             'utility_id_eia': [6452, 7801]},
    # 'AEP': {'utility_id_pudl': [29, 301, 144, 275, 162, 361, 7],
    #         'utility_id_eia': [733, 17698, 9324, 15474, 22053, 20521, 343]},
    # 'Entergy': {'utility_id_pudl': [107, 106, 311, 113, 110],
    #             'utility_id_eia': [11241, 814, 12465, 55937, 13478]},
    # 'Xcel': {'utility_id_pudl': [224, 302, 272, 11297],
    #          'utility_id_eia': [13781, 13780, 17718, 15466]}
}

In [3]:
specified_utilities = {
    'BHE': [12341, 14354, 13407, 17166],
    #'Southern':[7140, 195, 12686, 17622]
}

specified_years = [
    2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
] 

<a id='verify-tools'></a>
## 1) Output Override Tools
Run the following function and you'll find excel files called `<UTILITY>_fix_FERC-EIA_overrides.xlsx` in the `outputs/overrides` directory created based on the utility and year inputs you specified above. Read the [Override Instructions](https://docs.google.com/document/d/1nJfmUtbSN-RT5U2Z3rJKfOIhWsRFUPNxs9NKTes0SRA/edit#) to learn how to begin fixing/verifying the FERC-EIA connections.

In [7]:
generate_override_tools(pudl_out, rmi_out, specified_utilities, specified_years)

Generating inputs
Reading the FERC to EIA connection from /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/ferc1_eia.pkl.gz
Prepping FERC-EIA table
Reading the EIA plant-parts from /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/plant_parts_eia.pkl.gz
Prepping Plant Parts Table
Grabbing depreciation study output from /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/deprish.pkl.gz
Prepping Deprish Data
Developing outputs for BHE
Getting utility-year subset for ferc_eia
Getting utility-year subset for ppl
Getting utility-year subset for deprish
Outputing table subsets to tabs



<a id='upload-overrides'></a>
## 2) Upload changes to training data
When you've finished editing the `<UTILITY>_fix_FERC-EIA_overrides.xlsx` and want to add your changes to the official override csv, move your file to the directory called `add_to_training` and then run the following function. 

**Note:** If you have changed or marked TRUE any records that have already been overridden and included in the training data, you will want to set `expect_override_overrides = True`. Otherwise, the function will check to see if you have accidentally tampered with values that have already been matched.

Right now, the module points to a COPY of the training data so it doesn't override the official version. You'll need to change that later if you want to update the official version.

In [3]:
validate_and_add_to_training(
    pudl_out, rmi_out, expect_override_overrides=True
)

Reading the FERC to EIA connection from /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/ferc1_eia.pkl.gz
Reading the EIA plant-parts from /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/plant_parts_eia.pkl.gz
Processing fixes in BHE_fix_FERC-EIA_overrides_2020.xlsx
Validating overrides
Checking eia record id consistency for values that don't exist
Checking ferc record id consistency for values that don't exist
Checking for duplicate override ids
Checking for mismatched utility ids
Checking that year in override id matches report year
Combining all new overrides with existing training data
Found 119 new overrides
Adding record_id_ferc1 values with no EIA match to null_overrides csv
Found 2 new null matches


In [3]:
rmi_out.ferc1_to_eia(clobber=True)

FERC to EIA granular connection not found at /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/ferc1_eia.pkl.gz... Generating a new output.
Reading the EIA plant-parts from /Users/aesharpe/Desktop/Work/Catalyst_Coop/Repos/rmi-ferc1-eia/outputs/plant_parts_eia.pkl.gz
Preparing the FERC1 tables.
loading steam table
loading small gens table
loading hydro table
loading pumped storage table
prepping steam table
prepping hydro tables
combining all tables
Generated 168541 all candidate features.
Generated 3754 training candidate features.
We are about to test hyper parameters of the model while doing k-fold cross validation. This takes a few minutes....
train: newton-cg: c-1, cw-balanced, p-l2, l1-None
train: newton-cg: c-1, cw-balanced, p-l2, l1-None
train: newton-cg: c-1, cw-balanced, p-l2, l1-None
train: newton-cg: c-1, cw-balanced, p-none, l1-None
train: newton-cg: c-1, cw-balanced, p-none, l1-None
train: newton-cg: c-1, cw-balanced, p-none, l1-None
train: newton-cg: 

,record_id_ferc1,record_id_eia,match_type,plant_name_new,plant_part,report_year,report_date,ownership,plant_name_eia,plant_id_eia,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,ferc_acct_name,utility_id_eia,utility_id_pudl,true_gran,appro_part_label,appro_record_id_eia,record_count,fraction_owned,ownership_dupe,operational_status,operational_status_pudl,plant_id_pudl,total_fuel_cost_eia,fuel_cost_per_mmbtu_eia,net_generation_mwh_eia,capacity_mw_eia,capacity_factor_eia,total_mmbtu_eia,heat_rate_mmbtu_mwh_eia,fuel_type_code_pudl_eia,installation_year_eia,plant_part_id_eia,utility_id_ferc1,utility_name_ferc1,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor_ferc1,capacity_mw_ferc1,capex_equipment,capex_land,capex_per_mw,capex_structures,capex_total,construction_type,construction_year,installation_year_ferc1,net_generation_mwh_ferc1,not_water_limited_capacity_mw,opex_allowances,opex_boiler,opex_coolants,opex_electric,opex_engineering,opex_fuel,fuel_cost_per_mwh,opex_misc_power,opex_misc_steam,opex_nonfuel,opex_nonfuel_per_mwh,opex_operations,opex_per_mwh,opex_plant,opex_production_total,opex_rents,opex_steam,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,water_limited_capacity_mw,ferc_license_id,fuel_cost_per_mmbtu_ferc1,fuel_type,opex_maintenance,opex_total,plant_name_clean,total_cost_of_plant,capex_facilities,capex_roads,net_capacity_adverse_conditions_mw,net_capacity_favorable_conditions_mw,opex_dams,opex_generation_misc,opex_hydraulic,opex_misc_plant,opex_water_for_power,capex_equipment_electric,capex_equipment_misc,capex_wheels_turbines_generators,energy_used_for_pumping_mwh,net_load_mwh,opex_production_before_pumping,opex_pumped_storage,opex_pumping,total_fuel_cost_ferc1,total_mmbtu_ferc1,fuel_type_code_pudl_ferc1,plant_id_report_year,plant_id_report_year_util_id,heat_rate_mmbtu_mwh_ferc1,capex_wo_retirement_total,capex_total_shifted,capex_annual_addition,capex_annual_addition_rolling,capex_annual_per_mwh,capex_annual_per_mw,capex_annual_per_kw,capex_annual_per_mwh_rolling,capex_annual_per_mw_rolling,capex_annual_addition_gen_std,capex_annual_addition_gen_mean,capex_annual_addition_diff_mean
0,f1_steam_1994_12_1_0_1,NaN,NaN,<NA>,NaN,1994,1994-01-01,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,1,AEP Generating Company,1051,rockport unit 1,NaN,NaN,0.819843,650.0,490684127.0,6395551.0,894688.3,84467746.0,581547424.0,conventional,1984,1984.0,4668184.0,650.0,NaN,3185935.0,NaN,353599.0,427906.0,51694529.0,11.073799,1040610.0,781181.0,8300498.0,1.7781,1032559.0,12.9,631598.0,59995027.0,7559.0,442763.0,NaN,396788.0,NaN,650.0,NaN,NaN,steam,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527_1994,527_1994_7,NaN,581547424.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f1_steam_1995_12_1_0_1,NaN,NaN,<NA>,NaN,1995,1995-01-01,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,1,AEP Generating Company,1051,rockport u1 aeg,NaN,NaN,0.755040,650.0,490674427.0,6395551.0,894797.6,84548492.0,581618470.0,conventional,1984,1984.0,4299195.0,650.0,NaN,4320386.0,NaN,368314.0,542942.0,47711517.0,11.097779,1037095.0,721517.0,10016176.0,2.329779,1230673.0,13.4,1023977.0,57727693.0,5712.0,427845.0,NaN,337715.0,NaN,650.0,NaN,NaN,steam,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527_1995,527_1995_7,NaN,581618470.0,581547424.0,71046.0,NaN,0.016525,109.301538,0.109302,NaN,NaN,NaN,71046.0,0.0
2,f1_steam_1996_12_1_0_1,NaN,NaN,<NA>,NaN,1996,1996-01-01,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,1,AEP Generating Company,1051,rockpor